In [24]:
library(biomaRt)

In [ ]:
#get all protein coding genes
library(biomaRt)
grch37 = useMart(biomart="ENSEMBL_MART_ENSEMBL", host="grch37.ensembl.org", path="/biomart/martservice", dataset="hsapiens_gene_ensembl")

## get ensembl ID
helper <- function(str){
	unlist(strsplit(str, '\\.'))[1]
}

get_gene_index <- function(chr){
	glist = dir(paste0("/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/adjusted_expr1/chr", chr));
	tmp = sapply(glist, helper)
	unname(tmp)
}

pc_gene = list()
pc_num = c()
org_num = c()

for(chr in 1:22){
	esd = get_gene_index(chr)
	gene_info = getBM(filters= c("ensembl_gene_id","biotype"), attributes= c("ensembl_gene_id",'hgnc_symbol','chromosome_name'), values= list(esd,"protein_coding"), mart= grch37)
	pc_gene[[chr]] = gene_info
	pc_num = c(pc_num, nrow(gene_info))
	org_num = c(org_num, length(esd))
}
save(pc_gene, pc_num, org_num, file="/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/protein_coding.RData")

In [4]:
devtools::install_github("stephenturner/annotables")

from URL https://api.github.com/repos/stephenturner/annotables/zipball/master
Installing annotables
Installing tibble
Installing cli
Installing assertthat
'/ysm-gpfs/pi/zhao/Softwares/packages/R/lib64/R/bin/R' --no-site-file  \
  --no-environ --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/Rtmp811ZCs/devtools6db04ce83272/assertthat'  \
  --library='/gpfs/ysm/home/zy92/R/x86_64-pc-linux-gnu-library/3.5'  \
  --install-tests 

'/ysm-gpfs/pi/zhao/Softwares/packages/R/lib64/R/bin/R' --no-site-file  \
  --no-environ --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/Rtmp811ZCs/devtools6db04a5b572f/cli'  \
  --library='/gpfs/ysm/home/zy92/R/x86_64-pc-linux-gnu-library/3.5'  \
  --install-tests 

Installing fansi
'/ysm-gpfs/pi/zhao/Softwares/packages/R/lib64/R/bin/R' --no-site-file  \
  --no-environ --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/Rtmp811ZCs/devtools6db018457d9b/fansi'  \
  --library='/gpfs/ysm/home/zy92/R/x86_64-pc-linux-gnu-library/3.5'  \
  --install-tests 



In [7]:
library(annotables)
library(dplyr)

In [8]:
dim(grch38)

[1] 64428     9

In [9]:
head(grch38)

ensgene,entrez,symbol,chr,start,end,strand,biotype,description
ENSG00000000003,7105,TSPAN6,X,100627109,100639991,-1,protein_coding,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
ENSG00000000005,64102,TNMD,X,100584802,100599885,1,protein_coding,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
ENSG00000000419,8813,DPM1,20,50934867,50958555,-1,protein_coding,"dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]"
ENSG00000000457,57147,SCYL3,1,169849631,169894267,-1,protein_coding,SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
ENSG00000000460,55732,C1orf112,1,169662007,169854080,1,protein_coding,chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]
ENSG00000000938,2268,FGR,1,27612064,27635277,-1,protein_coding,"FGR proto-oncogene, Src family tyrosine kinase [Source:HGNC Symbol;Acc:HGNC:3697]"


In [12]:
grch38 %>% 
  #filter(biotype=="protein_coding") %>% 
  select(ensgene, entrez, symbol, chr, start, end, description) %>% 
  head


ensgene,entrez,symbol,chr,start,end,description
ENSG00000000003,7105,TSPAN6,X,100627109,100639991,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
ENSG00000000005,64102,TNMD,X,100584802,100599885,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
ENSG00000000419,8813,DPM1,20,50934867,50958555,"dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]"
ENSG00000000457,57147,SCYL3,1,169849631,169894267,SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
ENSG00000000460,55732,C1orf112,1,169662007,169854080,chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]
ENSG00000000938,2268,FGR,1,27612064,27635277,"FGR proto-oncogene, Src family tyrosine kinase [Source:HGNC Symbol;Acc:HGNC:3697]"


In [ ]:
grch38 %>% 
  filter(biotype=="protein_coding" & chr=="1") %>% 
  select(ensgene, symbol, chr, start, end, description) %>% 
  head %>% 
  pander::pandoc.table(split.table=100, justify="llllll", style="rmarkdown")

In [13]:
save(grch38, file = "./data_grch38.RData")

In [15]:
from_ref <- "symbol"
to_ref <- "ensgene"
id_df <- data.frame(raw_id = grch38$symbol[1:5])
colnames(id_df) <- from_ref

In [17]:
output_df <- id_df %>%
    left_join_(grch38, by = (from_ref = to_ref))

ERROR: Error in left_join_(., grch38, by = (from_ref = to_ref)): could not find function "left_join_"


In [24]:
by_str

ensgene 
"symbol"

In [32]:
by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
#by_str <- rlang::set_names(quo_name(to_ref), quo_name(from_ref))
output_df <- id_df %>%
    left_join(grch38, by = by_str) #%>%
    #select_(to_ref)


Warning message:
“Column `symbol` joining factor and character vector, coercing into character vector”

In [33]:
output_df

symbol,ensgene,entrez,chr,start,end,strand,biotype,description
TSPAN6,ENSG00000000003,7105,X,100627109,100639991,-1,protein_coding,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
TNMD,ENSG00000000005,64102,X,100584802,100599885,1,protein_coding,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
DPM1,ENSG00000000419,8813,20,50934867,50958555,-1,protein_coding,"dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]"
SCYL3,ENSG00000000457,57147,1,169849631,169894267,-1,protein_coding,SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
C1orf112,ENSG00000000460,55732,1,169662007,169854080,1,protein_coding,chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]


In [35]:
sum(complete.cases(grch38[,1:3]))

[1] 28457

In [ ]:
id_df <- data.frame(raw_id = grch38$symbol[1:5])
colnames(id_df) <- from_ref
by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
output_df <- id_df %>%
    left_join(annotable, by = by_str) #%>%
    #select_(to_ref)



In [44]:
class(grch38$ensgene)

[1] "character"

In [51]:
id_map <- function(id_vec, from_ref, to_ref, annotable = grch38) {
    id_df <- data.frame(raw_id = id_vec)
    colnames(id_df) <- from_ref
    id_df[, from_ref] <- as.character(id_df[, from_ref])
    by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
    output_df <- id_df %>%
        left_join(annotable, by = by_str) %>%
        select_(to_ref)
    return(output_df[,1])                              
    
} 

In [41]:
class(id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "ensgene"))

Warning message:
“Column `symbol` joining factor and character vector, coercing into character vector”

[1] "character"

In [52]:
id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "ensgene")

[1] "ENSG00000000457" "ENSG00000000460" "ENSG00000000938" "ENSG00000000971"
[5] "ENSG00000001036" "ENSG00000001084"

In [53]:
id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "entrez")

[1] 57147 55732  2268  3075  2519  2729

In [49]:
test_df <- head(grch38)

In [54]:
source("gene_id.R")

In [55]:
id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "ensgene")

[1] "ENSG00000000457" "ENSG00000000460" "ENSG00000000938" "ENSG00000000971"
[5] "ENSG00000001036" "ENSG00000001084"

In [56]:
id_map(grch38$ensgene[4:9], from_ref = "ensgene", to_ref =  "symbol")

[1] "SCYL3"    "C1orf112" "FGR"      "CFH"      "FUCA2"    "GCLC"

In [72]:
res_vec <- list.files(twas_res)
for (file_nam in res_vec) {
    cur_df <- data.table::fread(paste0(twas_res, file_nam))
    
}

In [73]:
cur_df

V1,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model
0,PSRC1,PSRC1,-6.697075,-0.297043657,2.126322e-11,4.037220e-02,0.5,1e-10,1e-10,18,18,18
1,LIPC,LIPC,-6.221472,-0.011994298,4.925108e-10,2.052324e+01,0.5,1e-10,1e-10,390,390,403
2,CELSR2,CELSR2,-5.925606,-1.750608832,3.111481e-09,9.379818e-04,0.5,1e-10,1e-10,30,31,31
3,ATG4D,ATG4D,NA,NA,NA,3.061570e-04,0.5,1e-10,1e-10,4,4,4
4,ZNF441,ZNF441,NA,NA,NA,3.274120e-06,0.5,1e-10,1e-10,11,11,11
5,QTRT1,QTRT1,NA,NA,NA,4.836572e-05,0.5,1e-10,1e-10,2,2,2
6,ZNF491,ZNF491,NA,NA,NA,1.449045e-07,0.5,1e-10,1e-10,34,34,34
7,PUM2,PUM2,-4.918214,-3.475233792,8.733733e-07,1.596023e-04,0.5,1e-10,1e-10,10,10,10
8,NLRC5,NLRC5,-4.654694,-3.016329855,3.244629e-06,1.780708e-04,0.5,1e-10,1e-10,18,19,19
9,CETP,CETP,-4.623897,-0.263001703,3.765973e-06,2.823351e-02,0.5,1e-10,1e-10,59,59,61


In [68]:
trait_str <- "FC"
test_type <- "single"
twas_dir <- paste0("/ysm-gpfs/pi/zhao-data/zy92/update_utmost/", trait_str, "/", test_type, "/")
res_vec <- list.files(twas_res)
for (file_nam in res_vec) {
    cur_df <- as.data.frame(data.table::fread(paste0(twas_dir, file_nam)))
    
}

In [64]:
res_vec <- list.files(twas_res)
for (file_nam in res_vec) {
    cur_df <- data.table::fread(paste0(twas_res, res_vec))
    
}

[1] "Liver_STARNET1.csv" "Liver.csv"

In [86]:


extract_twas_signature <- function(trait_str, test_type) {    

    res_vec <- c("Liver.csv", "Liver_STARNET1.csv")
    twas_dir <- paste0("/ysm-gpfs/pi/zhao-data/zy92/update_utmost/", trait_str, "/", test_type, "/")
    output_df <- data.frame(gene = character(),
                            zscore = character(), 
                            effect_size = character(),
                            pvalue = double(),
                            pvalue_adj = double(),
                            tissue = character(),
                            stringsAsFactors = FALSE) 
    
    for (file_nam in res_vec) {
        cur_df <- as.data.frame(data.table::fread(paste0(twas_dir, file_nam))) 
        cur_df <- cur_df[complete.cases(cur_df),]
        cur_df <- cur_df %>%
            select(gene, zscore, effect_size, pvalue)
        cur_df$pvalue_adj <- p.adjust(cur_df$pvalue, method = "fdr", n = length(cur_df$pvalue))
        cur_df$tissue <- unlist(stringr::str_split(file_nam, "\\."))[1]
        
        output_df <- rbind(output_df, cur_df)
            
    }
    return(output_df)
}

In [ ]:
pval

In [88]:
trait_str <- "FC"
test_type <- "single"
test_df <- extract_twas_signature(trait_str, test_type)
test_df <- test_df %>%
    arrange(pvalue_adj)
head(test_df)

gene,zscore,effect_size,pvalue,pvalue_adj,tissue
LIPC,-12.760447,-0.3921268,2.726337e-37,1.622988e-33,Liver_STARNET1
PSRC1,6.635514,0.5111294,3.233732e-11,9.625202e-08,Liver_STARNET1
PSRC1,-6.697075,-0.2970437,2.126322e-11,1.654278e-07,Liver
LIPC,-6.221472,-0.0119943,4.925108e-10,1.915867e-06,Liver
SORT1,6.075447,0.3889294,1.236431e-09,2.453492e-06,Liver_STARNET1
CELSR2,-5.925606,-1.7506088,3.111481e-09,8.069106e-06,Liver


In [89]:
test_df

gene,zscore,effect_size,pvalue,pvalue_adj,tissue
LIPC,-12.760447,-0.3921268,2.726337e-37,1.622988e-33,Liver_STARNET1
PSRC1,6.635514,0.5111294,3.233732e-11,9.625202e-08,Liver_STARNET1
PSRC1,-6.697075,-0.2970437,2.126322e-11,1.654278e-07,Liver
LIPC,-6.221472,-0.0119943,4.925108e-10,1.915867e-06,Liver
SORT1,6.075447,0.3889294,1.236431e-09,2.453492e-06,Liver_STARNET1
CELSR2,-5.925606,-1.7506088,3.111481e-09,8.069106e-06,Liver
CELSR2,5.802983,0.4866653,6.514532e-09,9.695252e-06,Liver_STARNET1
RP11-115J16.1,4.904522,0.1886279,9.365501e-07,1.115057e-03,Liver_STARNET1
PUM2,-4.918214,-3.4752338,8.733733e-07,1.698711e-03,Liver
NLRC5,-4.654694,-3.0163299,3.244629e-06,4.883212e-03,Liver


In [1]:
library("biomaRt")

In [32]:
missingdf <- as.data.frame(data.table::fread("../../twas_missing.TSS.info.txt"))
target_vec <- missingdf$gene

In [ ]:
grch37 = useMart(biomart="ENSEMBL_MART_ENSEMBL", host="grch37.ensembl.org", path="/biomart/martservice", dataset="hsapiens_gene_ensembl")



In [13]:
human = useMart("ensembl", dataset = "hsapiens_gene_ensembl")
res = getBM(attributes = c("hgnc_symbol","chromosome_name", "start_position"), 
       filters = "hgnc_symbol", 
       values = target_vec, 
       mart = human)

In [30]:
res_external = getBM(attributes = c("external_gene_name","chromosome_name", "start_position"), 
       filters = "external_gene_name", 
       values = target_vec, 
       mart = human)


In [31]:
dim(res)
dim(res_external)

[1] 43  3

[1] 43  3

In [26]:
write.table(res[complete.cases(res),], file = "../../match_table_biomart.txt", quote = F, row.names = F)

In [16]:
(res)

hgnc_symbol,chromosome_name,start_position
C19orf84,19,51388289
C5orf66,5,135033280
C8orf88,8,90958471
CCDC183,9,136796338
CCER2,19,38908980
CCER2,CHR_HG26_PATCH,38909016


In [31]:
head(target_vec)

ERROR: Error in head(target_vec): object 'target_vec' not found


In [17]:
dim(res)

[1] 43  3

In [10]:
length(target_vec)

[1] 262

In [22]:
res$chromosome_name <- as.numeric(res$chromosome_name)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [25]:
dim(res[complete.cases(res),])

[1] 34  3

In [21]:
sapply(res$chromosome_name, length)


19                 5                 8                 9 
                1                 1                 1                 1 
               19    CHR_HG26_PATCH                 4                14 
                1                 1                 1                 1 
                1                 7                10                17 
                1                 1                 1                 1 
               19                19                 4                 5 
                1                 1                 1                 1 
                7                 2                 2                 2 
                1                 1                 1                 1 
               10                11                 7                10 
                1                 1                 1                 1 
                2                17                 3                10 
                1                 1                 1                 1 
               11                12                10                22 
                1                 1                 1                 1 
 CHR_HG1311_PATCH CHR_HSCHR3_5_CTG3 CHR_HSCHR3_1_CTG3 CHR_HSCHR3_3_CTG3 
                1                 1                 1                 1 
CHR_HSCHR3_6_CTG3 CHR_HSCHR3_8_CTG3 CHR_HSCHR3_4_CTG3                 3 
                1                 1                 1                 1 
CHR_HSCHR3_7_CTG3                15                 7 
                1                 1                 1

### GTEx v6 genes

In [35]:
library(data.table)
gtex_gene <- fread("/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/gtexGene.txt")

In [36]:
colnames(gtex_gene)
head(gtex_gene)

[1] "#chrom"     "chromStart" "chromEnd"   "name"       "score"     
 [6] "strand"     "geneId"     "geneType"   "expCount"   "expScores"

#chrom,chromStart,chromEnd,name,score,strand,geneId,geneType,expCount,expScores
chr1,11868,13052,DDX11L1,15,+,ENSG00000223972.4,transcribed_unprocessed_pseudogene,53,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.239,0,0,0,0,"
chr1,14969,29806,WASH7P,427,-,ENSG00000227232.4,unprocessed_pseudogene,53,"6.886,6.083,4.729,5.91,6.371,6.007,8.768,4.202,4.455,4.64,10.097,10.619,6.108,5.037,5.018,4.808,4.543,4.495,5.576,4.57,8.275,4.707,2.55,9.091,9.885,8.17,7.392,7.735,5.353,7.124,8.617,3.426,2.375,7.669,3.826,7.094,6.365,3.263,10.723,10.507,4.843,9.193,13.25,11.635,11.771,8.641,10.448,6.522,9.313,10.304,9.987,9.067,6.12,"
chr1,29553,31109,RP11-34P13.3,63,+,ENSG00000243485.2,lincRNA,53,"0,0,0,0,0,0,0,0.041,0.056,0.094,0.054,0.051,0.078,0.097,0.05,0.058,0.1,0.062,0.017,0.038,0,0,0,0,0,0,0,0,0,0,0.023,0,0,0.043,0,0,0.022,0,0,0,0,0.024,0,0.039,0.055,0,0,0,0.366,0.017,0,0,0,"
chr1,34553,36081,FAM138A,0,-,ENSG00000237613.2,lincRNA,53,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"
chr1,52472,54936,OR4G4P,0,+,ENSG00000268020.2,unprocessed_pseudogene,53,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"
chr1,62947,63887,OR4G11P,0,+,ENSG00000240361.1,unprocessed_pseudogene,53,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"


In [37]:
res_startswith <- sapply(gtex_gene$name, function(x) startsWith(x,"LOC"))

In [40]:
gene_loc <- res_startswith[which(res_startswith)]

In [43]:
sum(res_startswith)

[1] 3

In [41]:
head(gene_loc)

LOC440461 LOC100421166    LOC401913 
        TRUE         TRUE         TRUE

In [29]:
grch37 = useMart(biomart="ENSEMBL_MART_ENSEMBL", host="grch37.ensembl.org", path="/biomart/martservice", dataset="hsapiens_gene_ensembl")

In [33]:
res = getBM(attributes = c("hgnc_symbol","chromosome_name", "start_position"), 
       filters = "hgnc_symbol", 
       values = target_vec, 
       mart = grch37)

In [36]:
res$chromosome_name <- as.numeric(res$chromosome_name)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [38]:
write.table(res[complete.cases(res),], file = "../../match_table_biomart_grch37.txt", quote = F, row.names = F)

In [39]:
getwd()

[1] "/gpfs/ysm/project/zhao/zy92/NASH/dr_multiomics/utilities"

### Database Gene List

In [13]:
library("RSQLite")
sqlite <- dbDriver("SQLite")
query <- function(...) dbGetQuery(db, ...)

weight_db_path <- "/ysm-gpfs/pi/zhao-data/zy92/UTMOST/sample_data/weight_db_GTEx_external/"

In [15]:
gene_list <- list()
fi <- list.files(weight_db_path, pattern = "*.db")
for (j in 1:length(fi))
{
    IRdisplay::display_html(paste0("INFO: ", "tissue ", j))
    dbname <- fi[j]
    fname <- unlist(strsplit(fi[j], split='.db', fixed=TRUE))[1]
    #dbname <- "TW_Brain_Cerebellum_0.5.db"
    if (length(gene_list[[fname]]) != 0) { next } 
    db <- dbConnect(sqlite, paste0(weight_db_path, dbname))
    #dbListTables(db)
    gene_list[[fname]] <- unique( query('select distinct(gene) from weights'))
} 

INFO: tissue 1

INFO: tissue 2

INFO: tissue 3

INFO: tissue 4

INFO: tissue 5

...

INFO: tissue 47

INFO: tissue 48

INFO: tissue 49

INFO: tissue 50

INFO: tissue 51

In [16]:
gene_unique_vec <- unique(unlist(gene_list))
length(gene_unique_vec)

[1] 22074

In [21]:
getwd()

[1] "/gpfs/ysm/project/zhao/zy92/NASH/dr_multiomics/utilities"

In [18]:
loc_idx <- ifelse(startsWith(gene_unique_vec, "LOC"), 1, 0)

In [22]:
write.table(sort(gene_unique_vec), file = "../../utmost_geneset.txt", row.names = F, quote = F)

In [ ]:
gene_list[[A]]

In [28]:
searchDatasets(mart = ensembl, pattern = "hsapiens")

ERROR: Error in searchDatasets(mart = ensembl, pattern = "hsapiens"): could not find function "searchDatasets"


In [8]:
dbname

[1] "glasso2_sig"